<a href="https://colab.research.google.com/github/IANovoselov/data_science/blob/main/ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print('hi')

hi
